# Single Transferable Vote
## Counting Tool using Droop and GSheets
Based on https://github.com/jklundell/droop

And https://github.com/custozza/droop_from_gspreadsheet


# Setup

In [ ]:
# !pip install requests 
!pip install requests pandas numpy

import requests
import pandas as pd
import numpy as np
import re
import subprocess

In [ ]:
!pip install -e deps/droop
# ignore egg not found error

In [181]:
# Update these:
SPREADSHEET_ID = '1ZaN_eCVB2kFFkmeShoGR--mXGxCig28xbw_maABR7zQ' # ID from your public spreadsheet URL
VOTE_FILE = 'vote.csv'
BLT_FILE = 'vote.blt'
PATH_TO_DROOP = "deps/droop"
PYTHON_COMMAND = "python"
ELECTION_TITLE = "Konferenz"
LOG_FILE = 'election.log'
ELECTED_FILE = 'elected.txt'
NUM_SEATS = 11

# Reading and Parsing

FETCH RESULT FROM SPREAD SHEET

In [182]:
url = f"https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/export?exportFormat=csv"

response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Get the content of the CSV
    csv_data = response.text

    # Process or print the CSV data as needed
    with open(VOTE_FILE, 'w', encoding="utf-8") as f:
        print(csv_data, file=f)
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")

CONVERT BALLOTS MATRIX TO SORTED CANDIDATES ID PER BALLOT

In [183]:
# # control terminal output
# lines = csv_data.split('\n')
# candidates = [[line.split(',')[0], line.strip().split(',')[1:]] for line in lines[1:]]
# longest_name_length = max([len(c[0]) for c in candidates])

# for i, (c, votes) in enumerate(candidates):
#     print(f"{i+1:2})", c.ljust(longest_name_length+5), '\t'.join(votes))


# CONVERT BALLOTS MATRIX TO SORTED CANDIDATES ID PER BALLOT

df = pd.read_csv(VOTE_FILE)
# number_of_winners = re.search(r'\d+', df.columns[0]).group()

candidates = df.values[:, 0]
ballots = df.values[:, 1:]

ballots_with_candidate_preferences = [
                                        [(candidate, pref) 
                                        for candidate, pref in enumerate(ballot) 
                                        if not np.isnan(pref)] 
                                        for ballot in ballots.T]

ballots_with_candidates_sorted_by_preference = [
                                        [candidate 
                                        for candidate, preference in sorted(ballot, key=lambda x: x[1])] 
                                        for ballot in ballots_with_candidate_preferences]


In [184]:
# # transform to create blt

# number_of_votes = len(candidates[0][1])
# number_of_candidates = len(candidates)

# ballot_matrix =  [[None] * number_of_candidates for _ in range(number_of_votes)]

# for ballot in range(number_of_votes): 
#     for candidate in range(number_of_candidates):
#         placement = candidates[candidate][1][ballot]
#         if placement:
#             placement = int(placement)
#             assert ballot_matrix[ballot][placement-1] == None
#             ballot_matrix[ballot][placement-1] = candidate + 1 # correcting the 0 indexing
#         else:
#             # empty slot
#             pass


# # remove empty fields
# ballot_matrix = [[placement for placement in ballot if placement] for ballot in ballot_matrix ]
    
# # print for control
# for i, ballot in enumerate(ballot_matrix):
#     names = [candidates[c-1][0] for c in ballot] # c-1 due to the shift of indexing of candidates
#     ballot = [str(b) for b in ballot]
#     print(i, " ".join(ballot), '|', ",".join(names))


WRITE BLT

In [ ]:
# pattern = r'\d+'

# # Search for the first number in the text to extract the number of winners
# # match = re.search(pattern, lines[0].split(',')[0])  
# # number_of_winners = match.group()
    
# with open(BLT_FILE, "w", encoding="utf-8") as f:
#     print(number_of_candidates, NUM_SEATS, file = f)
#     for i, ballot in enumerate(ballot_matrix):
#         ballot = [str(b) for b in ballot]
#         print(1, " ".join(ballot), 0, file=f)
#     print(0, '# end of ballots marker', file=f)
#     for i, candidate in enumerate(candidates):
#         print(f'"{candidate[0]}" #{i+1}', file=f)  # offset due to shift
#     print('"Sektion 8 Jahreskonferenz 2023"', file=f)

blt_ballots = "\n".join(
    [f"1 {' '.join([str(c+1) for c in ballot])} 0 #" + ", ".join(candidates[c] for c in ballot)
     for ballot in ballots_with_candidates_sorted_by_preference])

blt_candidates = "\n".join(f'"{candidate} #{i}"' for i, candidate in enumerate(candidates,1))

with open('vote.blt', 'w') as file:
    blt_content = f"""
    {len(candidates)} {NUM_SEATS}
    {blt_ballots}
    0 # end marker
    {blt_candidates}
    "{ELECTION_TITLE}" #Titel
    """
    file.write(blt_content)
    print(blt_content)

EXECUTE DROOP

In [ ]:
command = [PYTHON_COMMAND, PATH_TO_DROOP+'/Droop.py', "meek", BLT_FILE]
print(' '.join(command))
result = subprocess.run(command, capture_output=True, text=True)
with open(LOG_FILE, 'w') as file:
    file.write(result.stdout)

print('Ballots:', number_of_votes, 'Candidates:', number_of_candidates, 'Winners:', NUM_SEATS)
rounds = result.stdout.split('Round ')
election_pattern= r'Action: Elect: (?P<candidate>.*)'
electees_in_rounds = [(i, elected.group('candidate')) 
                      for i, r in enumerate(rounds[1:],1)
                      for elected in re.finditer(election_pattern, r) 
                       if elected ]

electees_in_round = "\n".join(f"{str(round).rjust(5)} {person}" for round, person in electees_in_rounds)
elected_text = f"""
ROUND NAME
{electees_in_round}
"""
with open(ELECTED_FILE, 'w') as file:
    file.write(elected_text)
    print(elected_text)

In [187]:
# Test call directly
# !python deps/droop/Droop.py meek vote.blt